In [1]:
import math
import OCC.Core.BRepBuilderAPI as BRepBuilderAPI
import OCC.Core.Geom as Geom
from OCC.Core.GeomAPI import GeomAPI_IntSS, GeomAPI_ExtremaCurveCurve, GeomAPI_ProjectPointOnCurve
from OCC.Core.Extrema import Extrema_POnCurv
from OCC.Core.gp import gp_Pnt, gp_Lin, gp_Ax1, gp_Dir, gp_Elips, gp_Ax2, gp_Ax3
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_MakeEdge, 
    BRepBuilderAPI_MakeVertex, 
    BRepBuilderAPI_MakeFace,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeShell,
    BRepBuilderAPI_MakeSolid   
)
from OCC.Core.BRep import BRep_Builder
from OCC.Core.TColgp import TColgp_Array1OfPnt
from OCC.Core.Geom import Geom_BezierCurve
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Core.TopoDS import TopoDS_Shell, TopoDS_Face, TopoDS_Solid

from OCC.Core.STEPControl import STEPControl_Writer, STEPControl_AsIs
from OCC.Core.IFSelect import IFSelect_RetDone

from OCC.Core.gp import (
    gp_Ax1,
    gp_Ax3,
    gp_Dir,
    gp_Pnt
)
from OCC.Core.Geom import (
    Geom_Plane,
    Geom_CylindricalSurface,
    Geom_ConicalSurface,
    Geom_SphericalSurface,
    Geom_ToroidalSurface,
    Geom_Line,
    Geom_Circle,
    Geom_Ellipse
)

from OCC.Core.BRep import (
    BRep_Builder
)

from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_MakeVertex,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace
)

import OCC.Core

In [2]:
from hybridbrep import HPart
import os

In [3]:
repbrep_path = '../../'
holybox2_path = os.path.join(repbrep_path, 'datasets', 'holybox', 'holey_box_angled.step')

target_part_path = holybox2_path
target_part = HPart(target_part_path, normalize=False).data
target_part_normalized = HPart(target_part_path, normalize=True).data

In [4]:
def make_surface(surface_type, params):
    origin = params[:3]
    normal = params[3:6]
    axis = params[6:9]
    ref_dir = params[9:12]
    radius = params[12]
    minor_radius = params[13]
    semi_angle = params[14]
    # TODO - orthonormalize coordinate system if necessary
    if surface_type == 0:
        cs = gp_Ax3(gp_Pnt(*origin), gp_Dir(*normal), gp_Dir(*ref_dir))
    else:
        cs = gp_Ax3(gp_Pnt(*origin), gp_Dir(*axis), gp_Dir(*ref_dir))

    if surface_type == 0:
        return Geom_Plane(cs)
    elif surface_type == 1:
        return Geom_CylindricalSurface(cs, radius)
    elif surface_type == 2:
        return Geom_ConicalSurface(cs, semi_angle, radius)
    elif surface_type == 3:
        return Geom_SphericalSurface(cs, radius)
    elif surface_type == 4:
        return Geom_ToroidalSurface(cs, radius, minor_radius)

def make_surfaces(faces):
    types = faces[:,:5].argmax(dim=1)
    parameters = faces[:,5:-1]
    flipped = faces[:,-1]

    surfaces = [
        make_surface(t.detach().cpu().tolist(), p.detach().cpu().tolist()) for t,p in zip(types, parameters)
    ]

    return surfaces

def make_curve(curve_type, params):
    origin = params[:3]
    direction = params[3:6]
    axis = params[6:9]
    x_dir = params[9:12]
    radius = params[12]
    minor_radius = params[13]

    if curve_type == 0:
        return Geom_Line(gp_Pnt(*origin), gp_Dir(*direction))
    elif curve_type == 1:
        return Geom_Circle(gp_Ax2(gp_Pnt(*origin), gp_Dir(*axis), gp_Dir(*x_dir)), radius)
    elif curve_type ==2:
        return Geom_Ellipse(gp_Ax2(gp_Pnt(*origin), gp_Dir(*axis), gp_Dir(*x_dir)), radius, minor_radius)

def make_curves(edges):
    types = edges[:,:3].argmax(dim=1)
    parameters = edges[:,3:-1]
    flipped = edges[:,-1]

    edges = [
        make_curve(t.detach().cpu().tolist(), p.detach().cpu().tolist()) for t,p in zip(types, parameters)
    ]

    return edges

def data_to_brep(data):
    data = data.detach().cpu()
    surfaces = make_surfaces(data.faces)
    curves = make_curves(data.edges)
    points = [gp_Pnt(*v.tolist()) for v in data.vertices]

    vertices = [BRepBuilderAPI_MakeVertex(p).Vertex() for p in points]

    edge_vertices = dict()
    for (vertex,edge), is_start in zip(data.vertex_to_edge.T.numpy(), data.vertex_to_edge_is_start.numpy()):
        verts = edge_vertices.get(edge,[])
        verts = [vertex] + verts if is_start else verts + [vertex]
        edge_vertices[edge] = verts

    edges = []
    for i,(curve,reversed) in enumerate(zip(curves, data.edges[:,-1].numpy().astype(bool))):
        verts = [vertices[j] for j in edge_vertices[i]]
        if len(verts) == 1: # Double up vertex if there is only one
            verts = verts*2
        edge_builder = BRepBuilderAPI_MakeEdge(curve, *verts)
        edge = edge_builder.Edge()
        if reversed:
            edge.Reverse()
        edges.append(edge)

    loop_edges = {}
    for (edge, loop), flipped in zip(data.edge_to_loop.T.numpy(), data.edge_to_loop_flipped.numpy().astype(bool)):
        l_edges = loop_edges.get(loop, [])
        e = (edges[edge], edge_vertices[edge])
        if flipped:
            e[1].reverse()
        l_edges.append(e)
        loop_edges[loop] = l_edges
    
    wires = []
    for loop, l_edges in loop_edges.items():
        # re-order edges if necessary
        if False > 0:
            ordered_edges = []
            ordered_edges.append(l_edges[0][0])
            next_start = l_edges[0][1][-1]
            used_edges = {0}
            for i,e in enumerate(l_edges):
                if i in used_edges:
                    continue
                if e[1][0] == next_start:
                    used_edges.add(i)
                    next_start = e[1][-1]
                    ordered_edges.append(e[0])
                    break
            assert len(used_edges) == len(l_edges)
        else:
            ordered_edges = [e[0] for e in l_edges]
        
        wire = BRepBuilderAPI_MakeWire()
        for e in ordered_edges:
            wire.Add(e)
        wires.append(wire.Wire())

    return vertices, edges, wires

wires, edges, vertices = data_to_brep(target_part)

In [5]:
def render_shapes(shapes):
    my_renderer = JupyterRenderer()
    for shape in shapes:
        my_renderer.DisplayShape(shape)
    return my_renderer.Display()

In [6]:
render_shapes(edges)

In [15]:
edges[0].Location()

<class 'TopLoc_Location'>

In [30]:
e = edges[0]
s = e.TShape()

In [31]:
s

<class 'TopoDS_TShape'>

In [35]:
import OCC.Core.ShapeAnalysis

In [36]:
sae = OCC.Core.ShapeAnalysis.ShapeAnalysis_Edge()

In [37]:
from OCC.Core.Geom import Geom_Curve

In [40]:
c = 0
cf = 0.0
cl = 0.0
orient = False
sae.Curve3d(e, c, orient)

TypeError: in method 'ShapeAnalysis_Edge_Curve3d', argument 3 of type 'opencascade::handle< Geom_Curve > &'

In [43]:
sae.HasCurve3d(e)

True

In [44]:
import OCC.Core.BRep

In [46]:
curve, start, end = OCC.Core.BRep.BRep_Tool.Curve(e, e.Location())

In [48]:
import numpy as np

In [60]:
import meshplot as mp
ppc = 100
pnts = []
colors = []
for e in edges:
    curve, start, end = OCC.Core.BRep.BRep_Tool.Curve(e, e.Location())
    for t in np.linspace(start, end, ppc):
        p = curve.Value(t)
        pnts.append([p.X(), p.Y(), p.Z()])
    cs = np.linspace(0,1,ppc)
    colors.append(cs)
pnts = np.array(pnts)
colors = np.concatenate(colors)
pl = mp.plot(target_part.V.numpy(), target_part.F.T.numpy(), return_plot=True)
pl.add_points(pnts, c=colors)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.019886…

1

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.019886…